<a href="https://colab.research.google.com/github/pyagoubi/Credit-Default-Prediction/blob/main/Amex_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing GPU support for LGBM

In [ ]:
!pip uninstall lightgbm

In [ ]:
!pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#set WD
import os
os.chdir('/content/drive/MyDrive/Amex/parquet')

# Load Libraries

In [ ]:
import pandas as pd, numpy as np # CPU libraries
import matplotlib.pyplot as plt, gc, os

from sklearn.model_selection import KFold
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import csv, itertools
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost

from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost import plot_importance
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import lightgbm as lgb

import joblib
import warnings
warnings.filterwarnings('ignore')
import itertools
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb


In [ ]:
#Global Variables

# VERSION NAME FOR SAVED MODEL FILES
VER = 20

# TRAIN RANDOM SEED
SEED = 42

# FILL NAN VALUE
NAN_VALUE = -127 # will fit in int8

# FOLDS PER MODEL
FOLDS = 5

NUM_PARTS = 4


TRAIN_PATH = 'train.parquet'
TEST_PATH = 'test.parquet'
TARGET_PATH = 'train_labels.csv'
SAVE_PATH = '/content/drive/MyDrive/Amex/parquet/XGB final/'
SUBMISSION_FILE_PATH = '/content/drive/MyDrive/Amex/parquet/sample_submission.csv'

# Preprocessing and Feature Engineering

In [ ]:
#read train or test file
def read_file(path = '', usecols = None):
    # LOAD DATAFRAME
    if usecols is not None: df = pd.read_parquet(path, columns=usecols)
    else: df = pd.read_parquet(path) #df = cudf.read_parquet(path)
    df['customer_ID'] = df['customer_ID'].str[-16:].apply(int, base =16)
    df.S_2 = pd.to_datetime( df.S_2 )
    print('shape of data:', df.shape)
    return df

#as we use the parquet dataset all nan values in int columns are replaced with -1, this function reverts that
def revertnan(df):
  df[df==-1] = np.nan 
  return df

#fill all na values
def fill_na(df, NAN_VALUE):
  df = df.fillna(NAN_VALUE)
  return df

#create feature that captures the number of statements a customer has in the dataset
def numberobs_feature(df):
  df['number_of_observations'] = df.groupby('customer_ID')['customer_ID'].transform('count')
  df.loc[df['B_33'].isnull() & (df.number_of_observations==1),'number_of_observations'] = 0.5
  return df

#subtraction of payment variables from balance variables
def afterpay(df):
  # compute "after pay" features
  for bcol in [f'B_{i}_last' for i in [11,14,17]]+['D_39_last','D_131_last']+[f'S_{i}_last' for i in [16,23]]:
    for pcol in ['P_2_last','P_3_last']:
      if bcol in df.columns:
        df[f'{bcol}-{pcol}'] = df[bcol] - df[pcol]
  return df

#classification of columns
def get_features(df):
  all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
  cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
  num_features = [col for col in all_cols if col not in cat_features]
  return all_cols, cat_features, num_features

#aggregating rows per customer and creating aggregation features
def agg_functions(df, num_features, cat_features, numberobs = False#, exclnullCols, 
                  #dummy_nan_col
                  ):
  
  test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'max', 'min', 'last', 'first'])
  print('num agg complete')  
  test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
  print('cat agg complete')

  df = pd.concat([test_num_agg, test_cat_agg],   #dummy_nan_col_agg, 
                   # test_nan_agg], 
                   axis=1)
  
  print('concat complete')
  df.columns = ['_'.join(x) for x in df.columns]

  for col in num_features:
    if col not in ['number_of_observations']:
      last = f'{col}_last'
      first = f'{col}_first'
      df[f'{col}_abschange'] = df[last] -df[first]  

  if numberobs ==True:
    print('drop numberobs')
    to_drop = ['number_of_observations_mean',  'number_of_observations_max','number_of_observations_std', 'number_of_observations_min', 'number_of_observations_first']
    df.drop(to_drop, axis = 1, inplace = True)
    df.rename(columns={'number_of_observations_last':'number_of_observations'}, inplace = True)
    print('drop numberobs complete')
  
  #drop all "first"-columns
  droplist = list(df.loc[:, df.columns.str.contains('first')].columns)
  df.drop(droplist, axis = 1, inplace = True)
  print('drop first complete')
  
  del test_num_agg, test_cat_agg
  _ = gc.collect()
  print('shape after engineering', df.shape )
  return df

#feature that contains the difference between last and mean
def add_meandev(df, num_features):
  
  for i in [f for f in num_features if f not in ['number_of_observations']]:
    last = f'{i}_last'
    mean = f'{i}_mean' 
    df[f'{i}_meandev'] = np.nan
    df.loc[(df[last] != np.nan), f'{i}_meandev'] = df[last] -df[mean]

  return df

#add labels to the train set
def add_targets(df, TARGET_PATH):
  # ADD TARGETS
  targets = pd.read_csv(TARGET_PATH)
  targets['customer_ID'] = targets['customer_ID'].str[-16:].apply(int, base =16)
  targets = targets.set_index('customer_ID')
  df = df.merge(targets, left_index=True, right_index=True, how='left', sort = True)
  df.target = df.target.astype('int8')
  del targets
  df = df.reset_index()
  return df

#first difference of the last statement for all numeric features
def get_difference(df, num_features, train_set = None, Part =None):
    df1 = []
    customer_ids = []
    for customer_id, cus in tqdm(df.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = cus[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    df1.set_index('customer_ID', inplace = True)
    if train_set == True: df1.to_parquet(f'{SAVE_PATH}diff_{VER}.parquet')
    elif train_set == False: df1.to_parquet(f'{SAVE_PATH}diff_test_{VER}_num{Part}.parquet')
    return df1

#use label encoding for categorical variables
def label_encoding(df, cat_cols):
  cat_cols = [f'{i}_last' for i in cat_cols]
  for col in cat_cols:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    return df

In [ ]:
#all preprocessing/fe functions
def preprocess(PATH = TRAIN_PATH, TARGET_PATH = TARGET_PATH, train_set = True, test = None, Part = None):
  if train_set == True:
    df = read_file(path = TRAIN_PATH)
  else:
    df = test
  print('read file complete')
  df = revertnan(df)
  print('revertnan complete')
  df = numberobs_feature(df)
  print('numberobs complete')
  all_cols, cat_features, num_features = get_features(df)
  print('get features complete')
  diff1 = get_difference(df, num_features, train_set = train_set, Part = Part)
  #if train_set == True: diff1 = pd.read_parquet(f'{SAVE_PATH}diff_12.parquet')
  #elif train_set == False: diff1 = pd.read_parquet(f'{SAVE_PATH}diff_test_12_num{Part}.parquet')
  print('get diff complete')
  df = agg_functions(df, num_features, cat_features, numberobs = True)
  df = df.merge(diff1, left_index=True, right_index=True, how='left') 
  del diff1
  _ = gc.collect()
  print('agg features complete')
  df = add_meandev(df, num_features)
  print('meandev complete')
  df = afterpay(df)
  print('afterpay complete')
  df = label_encoding(df, cat_cols=cat_features)
  print('label encoding complete')
  df = fill_na(df, NAN_VALUE)
  print('fillna complete')
  gc.collect()
  if train_set == True:
    df = add_targets(df, TARGET_PATH)
  return df

In [ ]:
train = preprocess(PATH = TRAIN_PATH)

shape of data: (5531451, 190)
read file complete
revertnan complete
numberobs complete
get features complete


  0%|          | 0/458913 [00:00<?, ?it/s]

get diff complete
num agg complete
cat agg complete
concat complete
drop numberobs
drop numberobs complete
drop first complete
shape after engineering (458913, 1096)
agg features complete
meandev complete
afterpay complete
label encoding complete
fillna complete


In [ ]:
#save training dataset
train.to_parquet(f'{SAVE_PATH}train_{VER}.parquet')

In [ ]:
#reading training dataset if saved
train = pd.read_parquet(f'{SAVE_PATH}train_{VER}.parquet')

In [ ]:
#getting categorical feature names as the model requires these, preprocessing fn could be improved by running this function before preprocessing, then no additional run is required
data = read_file(path = TRAIN_PATH)
all_cols, cat_features, num_features = get_features(data)
del data
cat_features = [f"{cf}_last" for cf in cat_features]

shape of data: (5531451, 190)


In [ ]:
#get all feature names
def get_feature_list(df):
  features = df.columns[1:-1]
  print(f'There are {len(features)} features!')
  return features

In [ ]:
features = get_feature_list(train)

There are 1465 features!


In [ ]:
#save feature names
def save_features(features):
  with open(f'{SAVE_PATH}features_V{VER}.csv', 'w') as csvfile:
    # creating a csv writer object
    writer = csv.writer(csvfile)    
    writer.writerow(features) 

In [ ]:
save_features(features)

In [ ]:
#read features if saved
features = pd.read_csv(f'{SAVE_PATH}features_V{VER}.csv')
features = pd.Index(features.columns)


# Train Model

In [ ]:
#competition metric
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
#LGB Parameters
def get_lgb_parameters():
  
  lgb_parms = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'dart',
        'seed': SEED,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40,
        'device_type': 'gpu'
        }
  
  return lgb_parms

In [ ]:
lgb_parms = get_lgb_parameters()

In [ ]:
#training the model 
def train_model(df, SEED=SEED, SAVE_PATH =SAVE_PATH, VER=VER, cat_features = cat_features):
  features_importance= pd.DataFrame({'Feature':[], 'Importance':[]})
  oof = []
  TRAIN_SUBSAMPLE = 1.0
  gc.collect()
  kaggle_metrics_folds =[]

  skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
  for fold,(train_idx, valid_idx) in enumerate(skf.split(
            df, df.target )):

    
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print(f'### Training with {int(TRAIN_SUBSAMPLE*100)}% fold data...')
    print('#'*25)
    
    # TRAIN, VALID, TEST FOR FOLD K
    X_train = df.loc[train_idx, features]
    y_train = df.loc[train_idx, 'target']

    X_valid = df.loc[valid_idx, features]
    y_valid = df.loc[valid_idx, 'target']
    

    lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = cat_features)
    lgb_valid = lgb.Dataset(X_valid, y_valid, categorical_feature = cat_features)

    print('datasets created')
    
    # TRAIN MODEL FOLD K

    model = lgb.train(
            params = lgb_parms,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 1500,
            verbose_eval = 300
            #feval = lgb_amex_metric
            )
    joblib.dump(model, f'{SAVE_PATH}lgbm_V{VER}_fold{fold}_seed{SEED}.pkl')

    fold_importance_df= pd.DataFrame({'Feature':[], 'Importance':[]})
    fold_importance_df['Feature']= features
    fold_importance_df['Importance']= model.feature_importance()
    fold_importance_df["fold"] = fold + 1
    features_importance = pd.concat([features_importance, fold_importance_df], axis=0)
            
    # INFER OOF FOLD K
    oof_preds = model.predict(df.loc[valid_idx, features], num_iteration=model.best_iteration)
    acc = amex_metric_mod(y_valid.values, oof_preds)
    print('Kaggle Metric =',acc,'\n')
    kaggle_metrics_folds.append(acc)
    
    # SAVE OOF
    df_pred = df.loc[valid_idx, ['customer_ID','target'] ].copy()
    df_pred['oof_pred'] = oof_preds
    oof.append( df_pred )
    
    del  X_train, y_train #,Xy_train,
    del X_valid, y_valid, model
    _ = gc.collect()
    
  print('#'*25)
  oof = pd.concat(oof,axis=0,ignore_index=True).set_index('customer_ID')
  acc = amex_metric_mod(oof.target.values, oof.oof_pred.values)
  print('OVERALL CV Kaggle Metric =',acc)
  print(kaggle_metrics_folds)

  oof.to_parquet(f'{SAVE_PATH}oof_V{VER}.csv')
  features_importance.to_parquet(f'{SAVE_PATH}importance_V{VER}.csv')
  return oof, features_importance

In [ ]:
oof, features_importance = train_model(train)

In [ ]:
oof = pd.read_parquet(f'{SAVE_PATH}oof_V{VER}.csv')

# Prepare and predict test data

In [ ]:
# CALCULATE SIZE OF EACH SEPARATE TEST PART
def get_rows(customers, test, NUM_PARTS = NUM_PARTS, verbose = ''):
    chunk = len(customers)//NUM_PARTS
    if verbose != '':
        print(f'We will process {verbose} data as {NUM_PARTS} separate parts.')
        print(f'There will be {chunk} customers in each part (except the last part).')
        print('Below are number of rows in each part:')
    rows = []

    for k in range(NUM_PARTS):
        if k==NUM_PARTS-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = test.loc[test.customer_ID.isin(cc)].shape[0]
        rows.append(s)
    if verbose != '': print( rows )
    return rows,chunk

In [ ]:
def get_rowsnumcust(TEST_PATH, NUM_PARTS=NUM_PARTS):
  print(f'Reading test data...')
  test = read_file(path = TEST_PATH, usecols = ['customer_ID','S_2'])
  customers = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
  rows,num_cust = get_rows(customers, test[['customer_ID']], NUM_PARTS = NUM_PARTS, verbose = 'test')
  return rows,num_cust,customers

In [ ]:
rows,num_cust, customers = get_rowsnumcust(TEST_PATH, NUM_PARTS=NUM_PARTS)

Reading test data...
shape of data: (11363762, 2)
We will process test data as 4 separate parts.
There will be 231155 customers in each part (except the last part).
Below are number of rows in each part:
[2841209, 2839857, 2842105, 2840591]


In [ ]:
# INFER TEST DATA IN PARTS
skip_rows = 0
skip_cust = 0
test_preds = []

for k in range(NUM_PARTS):
  # READ PART OF TEST DATA
  print(f'\nReading test data...')
  test = read_file(path = TEST_PATH)
  test = test.sort_index()
  test = test.iloc[skip_rows:skip_rows+rows[k]]
  skip_rows += rows[k]
  print(f'=> Test part {k+1} has shape', test.shape )
    
  # PROCESS AND FEATURE ENGINEER PART OF TEST DATA
  test = preprocess(train_set = False, test = test, Part =k)
  if k==NUM_PARTS-1: test = test.loc[customers[skip_cust:]]
  else: test = test.loc[customers[skip_cust:skip_cust+num_cust]]
  skip_cust += num_cust
 

  print('X_test complete')
  #dtest = xgb.DMatrix(data=X_test)
  print('dtest complete')
  #del test
  gc.collect()


  # INFER LGB MODELS ON TEST DATA
  model = joblib.load(f'{SAVE_PATH}lgbm_V{VER}_fold0_seed{SEED}.pkl')
  print('load model complete')
  preds = model.predict(test[features])
  print('preds complete')
  for f in range(1,FOLDS):
    del model
    gc.collect()
    model = joblib.load(f'{SAVE_PATH}lgbm_V{VER}_fold{f}_seed{SEED}.pkl')
    print(f'load {f} complete')
    preds += model.predict(test[features])
    print(f'preds {f} complete')
  preds /= FOLDS
  test_preds.append(preds)

  # CLEAN MEMORY
  del test, model
  _ = gc.collect()

test_preds = np.concatenate(test_preds)
test = pd.DataFrame(index=customers,data={'prediction':test_preds})
sub = pd.read_csv(SUBMISSION_FILE_PATH)[['customer_ID']]
sub['customer_ID_hash'] = sub['customer_ID'].str[-16:].apply(int, base =16)
sub = sub.set_index('customer_ID_hash')
sub = sub.merge(test[['prediction']], left_index=True, right_index=True, how='left')
sub = sub.reset_index(drop=True)

sub.to_csv(f'{SAVE_PATH}submission_xgb_v{VER}.csv',index=False)
print('Submission file shape is', sub.shape )
sub.head()